In [1]:
import numpy as np
import multiprocessing
import time
import sys


def sum_multiprocessing(A, start, end):
    s = 0
    for i in range(start, end):
        s += A[i]
    return s


def split_array_custom(array, num_parts):
    size = array.size
    ranges = []

    if num_parts == 1:
        # Un único rango que cubre todo el array
        ranges = [(0, size)]
    elif num_parts == 2:
        # Dos rangos que dividen el array en dos mitades
        mid = size // 2
        ranges = [(0, mid), (mid, size)]
    elif num_parts == 4:
        # Cuatro rangos que dividen el array en cuartos
        q1 = size // 4
        q2 = size // 2
        q3 = 3 * size // 4
        ranges = [(0, q1), (q1, q2), (q2, q3), (q3, size)]

    return ranges


if __name__ == "__main__":
    # Crear un array de números aleatorios
    #value = 5 * 10**7  # Tamaño del array
    value=int(sys.argv[1])
    X = np.random.rand(value)
    print(f"Tamaño del array: {X.size}\n")

    # Suma secuencial
    print("Suma secuencial usando un solo proceso")
    start_time = time.time()
    result = sum_multiprocessing(X, 0, X.size)  # Suma todo el array
    end_time = time.time()
    print(f"Resultado: {result}")
    print(f"Tiempo: {end_time - start_time:.6f} segundos\n")

    # Suma en paralelo
    for num_procs in [1, 2, 4]:
        print(f"Suma en paralelo usando {num_procs} procesos")

        # Obtener los rangos para los procesos
        ranges = split_array_custom(X, num_procs)
        # Preparar los argumentos como una lista de tuplas (array, start, end)
        args = [(X, start, end) for start, end in ranges]

        start_time = time.time()
        with multiprocessing.Pool(processes=num_procs) as pool:
            # Usar starmap para ejecutar sum_multiprocessing con los rangos
            partial_results = pool.starmap(sum_multiprocessing, args)
            total_sum = sum(partial_results)  # Sumar los resultados parciales
        end_time = time.time()

        print(f"Resultado: {total_sum}")
        print(f"Tiempo: {end_time - start_time:.6f} segundos\n")


Tamaño del array: 50000000

Suma secuencial usando un solo proceso
Resultado: 25003558.34750778
Tiempo: 4.902518 segundos

Suma en paralelo usando 1 procesos
Resultado: 25003558.34750778
Tiempo: 5.750307 segundos

Suma en paralelo usando 2 procesos
Resultado: 25003558.347505853
Tiempo: 3.198278 segundos

Suma en paralelo usando 4 procesos
Resultado: 25003558.34750394
Tiempo: 2.572638 segundos



### Resultado obtenido 

Tamaño del array: 50000000

Suma secuencial usando un solo proceso
Resultado: 24998308.129806932
Tiempo: 4.297643 segundos

Suma en paralelo usando 1 procesos
Resultado: 24998308.129806932
Tiempo: 4.764542 segundos

Suma en paralelo usando 2 procesos
Resultado: 24998308.12981318
Tiempo: 2.928582 segundos

Suma en paralelo usando 4 procesos
Resultado: 24998308.12981929
Tiempo: 2.503165 segundos

### Comentarios

En los resultados obtenidos podemos ver que la suma secuencial es la más lenta, con un tiempo de 4.29 segundos. 
Por otro lado, al usar 1 proceso en paralelo no mejora el tiempo, ya que incurre en la sobrecarga del manejo de procesos.
Seguidamente, al usar 2 procesos, el tiempo mejora significativamente (2.93 segundos).
Finalmente, con 4 procesos, mejora aún más (2.5 segundos). Esto demuestra que dividir el trabajo en múltiples procesos acelera la ejecución, aunque el beneficio disminuye al aumentar los procesos debido a la sobrecarga de coordinación entre ellos.